# Sentiment Analysis con KNN
## Clasificador en C++ 💪💪
Vamos a probar a nuestro bichito


Definir los path al ejecutable de python 3.6 y sus librerías,
de acuerdo al virtual env que estén corriendo.

In [1]:
!cd .. && git submodule init
!cd .. && git submodule update
!cd .. && mkdir build
!cd ../build/ && rm -rf *
!cd ../build && cmake \
  -DPYTHON_EXECUTABLE="$(which python)" \
  -DCMAKE_BUILD_TYPE=Debug ..
!cd ../build && make install

mkdir: build: File exists
-- The C compiler identification is AppleClang 10.0.1.10010046
-- The CXX compiler identification is AppleClang 10.0.1.10010046
-- Check for working C compiler: /Library/Developer/CommandLineTools/usr/bin/cc
-- Check for working C compiler: /Library/Developer/CommandLineTools/usr/bin/cc -- works
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Detecting C compile features
-- Detecting C compile features - done
-- Check for working CXX compiler: /Library/Developer/CommandLineTools/usr/bin/c++
-- Check for working CXX compiler: /Library/Developer/CommandLineTools/usr/bin/c++ -- works
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Detecting CXX compile features
-- Detecting CXX compile features - done
Debug mode
-- Found PythonInterp: /anaconda3/bin/python (found version "3.7.3") 
-- Found PythonLibs: /anaconda3/lib/libpython3.7m.dylib
-- pybind11 v2.3.dev0
-- Performing Test HAS_FLTO
-- Performing Test 

In [2]:
# Verifico la correcta instalación. Si no falla el import está OK
!pwd
!python --version
import sentiment


/Users/vpomsztein/Documents/PrivateRepositories/SentimentAnalysisMetNum/notebooks
Python 3.7.3


In [3]:
%load_ext autoreload
%autoreload 2
import pandas as pd

!cd ../data && tar -xvf *.tgz

df = pd.read_csv("../data/imdb_small.csv", index_col=0)

print("Cantidad de documentos: {}".format(df.shape[0]))

x ./._imdb_small.csv
x imdb_small.csv
Cantidad de documentos: 12500


In [4]:
df.describe()

,type,review,label,file
count,12500,12500,12500,12500
unique,2,12469,2,12085
top,test,The fact that this cruddy series could elicit ...,neg,351_10.txt
freq,6275,2,6322,2


In [5]:
text_train = df[df.type == 'train']["review"]
label_train = df[df.type == 'train']["label"]

text_test = df[df.type == 'test']["review"]
label_test = df[df.type == 'test']["label"]

print("Cantidad de instancias de entrenamiento = {}".format(len(text_train)))
print("Cantidad de instancias de test = {}".format(len(text_test)))



Cantidad de instancias de entrenamiento = 6225
Cantidad de instancias de test = 6275


In [6]:
from __future__ import division
print("Class balance : {} pos {} neg".format(
    (label_train == 'pos').sum() / label_train.shape[0], 
    (label_train == 'neg').sum() / label_train.shape[0]
))


Class balance : 0.49493975903614457 pos 0.5050602409638554 neg


In [7]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(max_df=0.90, min_df=0.01, max_features=5000)

vectorizer.fit(text_train)

X_train, y_train = vectorizer.transform(text_train), (label_train == 'pos').values
X_test, y_test = vectorizer.transform(text_test), (label_test == 'pos').values

In [ ]:
# import numpy as np
# from numpy import linalg as LA

# A = np.matrix([[1, 0, 1],[0, 2, 0],[1, 0, 3]])
# w, v = sentiment.get_first_eigenvalues(A, 3, num_iter=50000)
# print(w)
# print(v)

# for i in range(0, w.size):
#     x = v.transpose()[i]
#     print(np.absolute(A @ x) - np.absolute(w[i] * x))

# print('-------------')

# w, v = LA.eig(A)
# print(w)
# print(v)

In [ ]:
import sentiment

# Esta celda puede ser ejecutada sólo cuando se quiere probar PCA
pca = sentiment.PCA(100)
pca.fit(X_train[:500])
pca_x_train = pca.transform(X_train[:500])
pca_y_train = pca.transform(y_train[:500])

pca_x_test = pca.transform(X_test[:500])
pca_y_test = pca.transform(y_test[:500])

In [15]:
%%time
from sklearn.metrics import accuracy_score

# KNN only (PCA disabled):
clf = sentiment.KNNClassifier(25)
clf.fit(X_train.copy(), y_train.copy())

y_pred = clf.predict(X_test[:50].copy())
acc = accuracy_score(y_test[:50], y_pred[:50])

# PCA enabled:
# clf = sentiment.KNNClassifier(100)
# clf.fit(pca_x_train, y_train)

# y_pred = clf.predict(pca_x_test[:100])
# acc = accuracy_score(y_test[:100], y_pred[:100])

print("Accuracy: {}".format(acc))

Accuracy: 0.68
CPU times: user 42.9 s, sys: 990 ms, total: 43.9 s
Wall time: 45.3 s
